In [1]:
import os

In [2]:
%pwd

'c:\\Users\\lenovo\\OneDrive\\Desktop\\End to end Projects\\Machine-Learning-Project-with-ML-Flow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\lenovo\\OneDrive\\Desktop\\End to end Projects\\Machine-Learning-Project-with-ML-Flow'

In [5]:
# update entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [7]:
from src.mlProject.constant import *
from src.mlProject.utils.common import read_yaml,create_directories

In [8]:
# update configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion   
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
            
        )
        
        return data_ingestion_config
        

In [11]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size 

In [12]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers =  request.urlretrieve(
                
                url = self.config.source_URL,
                filename = self.config.local_data_file
                
                )
            logger.info(f"{filename} downloaded! with headers : {headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}")   
       
    def extract_zip_file(self):
        '''
        Extracts zip file into the data directory
        
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)  
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
                   

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e    
    

[2024-07-03 02:03:43,586 : INFO : common : Directory created : artifacts]
[2024-07-03 02:03:43,589 : INFO : common : Directory created : artifacts/data_ingestion]


[2024-07-03 02:03:44,833 : INFO : 4041011692 : artifacts/data_ingestion/data.zip downloaded! with headers : Connection: close
Content-Length: 87168
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "5ae5ec064d7b52cde534dabc74558ffc8d23a9de45faef320dceee43790a1ed8"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8F30:3B565:11002:187A1:66846426
Accept-Ranges: bytes
Date: Tue, 02 Jul 2024 20:33:44 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4726-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1719952424.490663,VS0,VE341
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 4dff82f3fc3a7f41f3e4fb0aed5f3cff0493f2c8
Expires: Tue, 02 Jul 2024 20:38:44 GMT
Source-Age: 0

]
